# Amoritized Inference for source-level variables

In [ ]:
using Gen;
using Statistics: mean, std;
using LinearAlgebra: dot;
using StatsFuns: logsumexp;
using PyPlot
include("./time_helpers.jl")
include("./extra_distributions.jl")
include("./gaussian_helpers.jl")

In [ ]:
source_params, steps, gtg_params, obs_noise = include("./base_params.jl")
scene_duration = 2.0

### Wait proposal

In [ ]:
@gen function wait_model()
    
    ##Element spacing 
    tp_latents = Dict(:wait=>Dict()) #just do wait for now
    tp_params = source_params["tp"]
    for tp_type in keys(tp_latents)
        hyperpriors = tp_params[String(tp_type)]
        for latent in keys(hyperpriors)
            hyperprior = hyperpriors[latent]; syml = Symbol(latent)
            tp_latents[tp_type][syml] = @trace(hyperprior["dist"](hyperprior["args"]...), tp_type => syml)
        end
        tp_latents[tp_type][:args] = (tp_latents[tp_type][:a], tp_latents[tp_type][:mu]/tp_latents[tp_type][:a]) #a, b for gamma
    end
    
    ne_params = source_params["n_elements"]
    n_elements = ne_params["type"] == "max" ? 
        @trace(uniform_discrete(1, ne_params["val"]),:n_elements) : 
        @trace(geometric(ne_params["val"]),:n_elements)    
    
    waits = []
    for element_idx = 1:n_elements
        wait = element_idx == 1 ? @trace(uniform(0, scene_duration), (:element,element_idx)=>:wait) : 
            @trace(gamma(tp_latents[:wait][:args]...), (:element,element_idx)=>:wait)
        push!(waits, wait)
    end
    
    return waits
    
end


function data_generator()
    
    trace = simulate(wait_model, ())
    waits = get_retval(trace)
    
    constraints = choicemap()
    constraints[:wait => :mu] = trace[:wait => :mu]
    constraints[:wait => :a] = trace[:wait => :a]
    
    return ((waits,), constraints)
    
end


@gen function custom_dest_proposal_trainable(waits)
    
    #B_Qparameter
    @param B_mu_mu::Vector{Float64}
    @param B_logsigma_mu::Vector{Float64}
    @param B_mu_alpha::Vector{Float64}
    @param B_logsigma_alpha::Vector{Float64}
    @param log_std_replace::Float64

    n_w = length(waits)
    mu_w = mean(waits)
    sigma_w = n_w > 1.0 ? std(waits) : exp(log_std_replace)
    X = [mu_w, sigma_w, n_w, 1]
    
    mu_mu = dot(X, B_mu_mu)
    logsigma_mu = dot(X, B_logsigma_mu)
    mu_alpha = dot(X, B_mu_alpha)
    logsigma_alpha = dot(X, B_logsigma_alpha)
    
    @trace(log_normal(mu_mu,exp(logsigma_mu)),:wait => :mu)
    @trace(log_normal(mu_alpha,exp(logsigma_alpha)),:wait => :a)
    
    return nothing
    
end

for p in [:B_mu_mu, :B_logsigma_mu, :B_mu_alpha, :B_logsigma_alpha]
    Gen.init_param!(custom_dest_proposal_trainable, p, zeros(4))
end
Gen.init_param!(custom_dest_proposal_trainable, :log_std_replace,0.)
#Gen.ADAM(0.001, 0.9, 0.999, 1e-08)
update = Gen.ParamUpdate(Gen.FixedStepGradientDescent(0.00001), custom_dest_proposal_trainable);
scores = Gen.train!(custom_dest_proposal_trainable, data_generator, update,
    num_epoch=100, epoch_size=1000, num_minibatch=1, 
    minibatch_size=1000, evaluation_size=100, verbose=true);

plot(scores)
xlabel("Iterations of stochastic gradient descent")
ylabel("Estimate of expected conditional log probability density");
title("Wait")

println("Wait Q params:")
println("B_mu_mu: ", Gen.get_param(custom_dest_proposal_trainable, :B_mu_mu))
println("B_logsigma_mu: ", Gen.get_param(custom_dest_proposal_trainable, :B_logsigma_mu))
println("B_mu_alpha: ", Gen.get_param(custom_dest_proposal_trainable, :B_mu_alpha))
println("B_logsigma_alpha: ", Gen.get_param(custom_dest_proposal_trainable, :B_logsigma_alpha))
println("log_std_replace: ", Gen.get_param(custom_dest_proposal_trainable, :log_std_replace))

### Duration proposal

In [ ]:
@gen function durminusmin_model()
    
    ##Element spacing 
    tp_latents = Dict(:dur_minus_min=>Dict()) #just do wait for now
    tp_params = source_params["tp"]
    for tp_type in keys(tp_latents)
        hyperpriors = tp_params[String(tp_type)]
        for latent in keys(hyperpriors)
            hyperprior = hyperpriors[latent]; syml = Symbol(latent)
            tp_latents[tp_type][syml] = @trace(hyperprior["dist"](hyperprior["args"]...), tp_type => syml)
        end
        tp_latents[tp_type][:args] = (tp_latents[tp_type][:a], tp_latents[tp_type][:mu]/tp_latents[tp_type][:a]) #a, b for gamma
    end
    
    ne_params = source_params["n_elements"]
    n_elements = ne_params["type"] == "max" ? 
        @trace(uniform_discrete(1, ne_params["val"]),:n_elements) : 
        @trace(geometric(ne_params["val"]),:n_elements)    
    
    dur_minus_mins = []
    for element_idx = 1:n_elements
        dur_minus_min = @trace(truncated_gamma(tp_latents[:dur_minus_min][:args]..., source_params["duration_limit"]), (:element,element_idx)=>:dur_minus_min); 
        push!(dur_minus_mins, dur_minus_min)
    end
    
    return dur_minus_mins
    
end


function data_generator()
    
    trace = simulate(durminusmin_model, ())
    dur_minus_mins = get_retval(trace)
    
    constraints = choicemap()
    constraints[:dur_minus_min => :mu] = trace[:dur_minus_min => :mu]
    constraints[:dur_minus_min => :a] = trace[:dur_minus_min => :a]
    
    return ((dur_minus_mins,), constraints)
    
end

@gen function trainable_durminusmin_proposal(dur_minus_mins)
    
    #B_Qparameter
    @param B_mu_mu::Vector{Float64}
    @param B_logsigma_mu::Vector{Float64}
    @param B_mu_alpha::Vector{Float64}
    @param B_logsigma_alpha::Vector{Float64}
    @param log_std_replace::Float64

    n_d = length(dur_minus_mins)
    mu_d = mean(dur_minus_mins)
    sigma_d = n_d > 1.0 ? std(dur_minus_mins) : exp(log_std_replace)
    X = [mu_d, sigma_d, n_d, 1]
    
    mu_mu = dot(X, B_mu_mu)
    logsigma_mu = dot(X, B_logsigma_mu)
    mu_alpha = dot(X, B_mu_alpha)
    logsigma_alpha = dot(X, B_logsigma_alpha)
    
    @trace(log_normal(mu_mu,exp(logsigma_mu)),:dur_minus_min => :mu)
    @trace(log_normal(mu_alpha,exp(logsigma_alpha)),:dur_minus_min => :a)
    
    return nothing
    
end

for p in [:B_mu_mu, :B_logsigma_mu, :B_mu_alpha, :B_logsigma_alpha]
    Gen.init_param!(trainable_durminusmin_proposal, p, zeros(4))
end
Gen.init_param!(trainable_durminusmin_proposal, :log_std_replace,0.)
#Gen.ADAM(0.001, 0.9, 0.999, 1e-08)
update = Gen.ParamUpdate(Gen.FixedStepGradientDescent(0.00001), trainable_durminusmin_proposal);
scores = Gen.train!(trainable_durminusmin_proposal, data_generator, update,
    num_epoch=100, epoch_size=1000, num_minibatch=1, 
    minibatch_size=1000, evaluation_size=100, verbose=false);

plot(scores)
xlabel("Iterations of stochastic gradient descent")
ylabel("Estimate of expected conditional log probability density");
title("Dur minus min")
println("Dur_minus_min Q params:")
println("B_mu_mu: ", Gen.get_param(custom_dest_proposal_trainable, :B_mu_mu))
println("B_logsigma_mu: ", Gen.get_param(custom_dest_proposal_trainable, :B_logsigma_mu))
println("B_mu_alpha: ", Gen.get_param(custom_dest_proposal_trainable, :B_mu_alpha))
println("B_logsigma_alpha: ", Gen.get_param(custom_dest_proposal_trainable, :B_logsigma_alpha))
println("log_std_replace: ", Gen.get_param(custom_dest_proposal_trainable, :log_std_replace))

In [ ]:
@gen function gp_model(source_type, gp_type)
    
    ##GPs 
    gp_params = source_params["gp"]
    gp_latents = Dict(gp_type => Dict())
    for gp_type in keys(gp_latents)
        hyperpriors = gp_type === :erb ? gp_params["erb"] : 
            ((source_type == "noise" || source_type == "harmonic") ? gp_params["amp"]["2D"] : gp_params["amp"]["1D"] )
        for latent in keys(hyperpriors)
            hyperprior = hyperpriors[latent]; syml = Symbol(latent)
            gp_latents[gp_type][syml] = @trace(hyperprior["dist"](hyperprior["args"]...), gp_type => syml)
        end
    end
    
    ##Element spacing 
    tp_latents = Dict(:wait=>Dict(),:dur_minus_min=>Dict()) 
    tp_params = source_params["tp"]
    for tp_type in keys(tp_latents)
        hyperpriors = tp_params[String(tp_type)]
        for latent in keys(hyperpriors)
            hyperprior = hyperpriors[latent]; syml = Symbol(latent)
            tp_latents[tp_type][syml] = @trace(hyperprior["dist"](hyperprior["args"]...), tp_type => syml)
        end
        tp_latents[tp_type][:args] = (tp_latents[tp_type][:a], tp_latents[tp_type][:mu]/tp_latents[tp_type][:a]) #a, b for gamma
    end
    
    ne_params = source_params["n_elements"]
    n_elements = ne_params["type"] == "max" ? 
        @trace(uniform_discrete(1, ne_params["val"]),:n_elements) : 
        @trace(geometric(ne_params["val"]),:n_elements)    
    
    waits = []
    dur_minus_mins = []
    prev_gps = Dict(:x => [], gp_type => [])
    time_so_far = 0.0;
    for element_idx = 1:n_elements
        wait = element_idx == 1 ? @trace(uniform(0, scene_duration), (:element,element_idx)=>:wait) : 
            @trace(gamma(tp_latents[:wait][:args]...), (:element,element_idx)=>:wait)
        dur_minus_min = @trace(truncated_gamma(tp_latents[:dur_minus_min][:args]..., source_params["duration_limit"]), (:element,element_idx)=>:dur_minus_min); 

        push!(waits, wait)
        push!(dur_minus_mins, dur_minus_min)
        duration = dur_minus_min + steps["min"]; onset = time_so_far + wait; 
        time_so_far = onset + duration; element_timing = [onset, time_so_far]

        ## Define points at which the GPs should be sampled
        if gp_type === :erb || (source_type == "tone" && gp_type === :amp)
            x = get_element_gp_times(element_timing, steps["t"])
        elseif gp_type === :amp && (source_type == "noise"  || source_type == "harmonic")
            x, ts, fs = get_gp_spectrotemporal(element_timing, steps, audio_sr)
        end

        mu, cov = element_idx == 1 ? get_mu_cov(x, gp_latents[gp_type]) : 
                get_cond_mu_cov(x, prev_gps[:x], prev_gps[gp_type], gp_latents[gp_type])
        element_gp = @trace(mvnormal(mu, cov), (:element, element_idx) => gp_type)
        
        append!(prev_gps[:x],x) #could also be push
        append!(prev_gps[gp_type],element_gp)
                
    end
    
    return waits, dur_minus_mins, prev_gps
    
end

source_type="tone";gp_type=:amp;
function data_generator()
    
    trace = simulate(gp_model, (source_type, gp_type))
    waits, dur_minus_mins, prev_gps = get_retval(trace)
    
    constraints = choicemap()
    d = gp_type == :erb ? source_params["gp"]["erb"] : (source_type == "tone" ? source_params["gp"]["amp"]["1D"] : source_params["gp"]["amp"]["2D"]) 
    for k in keys(d)
        constraints[gp_type => Symbol(k)] = trace[gp_type => Symbol(k)]
    end
    
    return ((prev_gps,), constraints)
    
end

In [ ]:
function trainable_amp1D_proposal(gps)

    #Mu and Sigma of GP
    @param B_mu_mu::Vector{Float64}
    @param B_logsigma_mu::Vector{Float64}
    @param B_mu_sigma::Vector{Float64}
    @param B_logsigma_sigma::Vector{Float64}
    @param log_std::Float64
    
    mu_d = means(gps[:amp])
    sigma_d = (length(gps[:amp]) || all(y->y==gps[:amp][1], gps[:amp])) > 1 ? std(gps[:amp]) : exp(log_std_replace)
    X = [mu_d, sigma_d, 1]
    
    mu_mu =dot(X, B_mu_mu)
    logsigma_mu = dot(X, B_logsigma_mu)
    mu_sigma = dot(X, B_mu_sigma)
    logsigma_sigma = dot(X, B_logsigma_sigma)
    
    gp_mu = @trace(normal(mu_mu,exp(logsigma_mu)),:amp => :mu)
    gp_sigma = @trace(log_normal(mu_sigma, exp(logsigma_sigma), :amp => :sigma))
    
    #Temporal lengthscale of GP
    @param W_mu_scale::Vector{Float64}
    @param C_mu_scale::Float64
    @param replace_mu_scale::Float64
    
    @param W_logsigma_scale::Vector{Float64}
    @param W_logsigma_scale::Float64
    @param replace_logsigma_scale::Float64
    
    @param mu_times::Vector{Float64}
    @param logsigma_times::Vector{Float64}
    
    if length(gps[:x]) > 1
        
        n = sum(1:length(gps[:x])-1)
        delta_ts = Vector{Float64}(nothing, n)
        delta_ys = Vector{Float64}(nothing, n)
        rs = Vector{Float64}(nothing, n)
        ps = Array{Float64}(nothing, n, 3)
        k = 1
        for i = 1:length(gps[:x])
            for j = (i+1):length(gps[:x])
                delta_ts[k] = gps[:x][j] - gps[:x][i]
                delta_ys[k] = abs(gps[:amp][j] - gps[:amp][i])
                rs[k] = delta_ys[k]/(delta_ts[k]*gp_sigma)
                ps[k,:] = logsumexp([logpdf(normal, delta_ts[k], mu_times[g],exp(logsigma_times[g])) for g = 1:3])
                k += 1
            end
        end
        
        mu_scale = rs*p*W_mu_scale + C_mu_scale #mu_scale(1)=rs(n,)*ps(n,3)*W(3,)
        logsigma_scale = rs*p*w_logsigma_scale + C_logsigma_scale 
        
    else
        
        mu_scale = replace_mu_scale
        logsigma_scale = replace_logsigma_scale
        
    end

    @trace(log_normal(mu_scale, exp(logsigma_scale)), :amp=>:scale)
    
    ## epislon, local noise parameter of GP 
    @param replace_mu_noise::Float64
    @param replace_logsigma_noise::Float64
    
    @trace(log_normal(mu_noise, exp(logsigma_noise)), :amp=>:noise)
    
end
